In [1]:
import os
import requests
from uber_time_estimation.configs.settings import ROOT_DIR
# Set the base URL
BASE_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/"
SAVE_DIR = ROOT_DIR / "data" / "nyc_tlc_yellow_taxi_data"

# Years and months to download (customize as needed)
years = range(2009, 2025)  # 2009 to 2024
months = range(1, 13)  # Jan to Dec

# Create directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)

from tqdm.notebook import tqdm  # Use notebook-friendly tqdm

# Loop over each year and month with nested progress bars
for year in tqdm(years, desc="Years", unit="year", position=0):
    for month in tqdm(months, desc="Months", unit="month", leave=False, position=1):
        file_name = f"yellow_tripdata_{year}-{month:02}.parquet"
        file_url = BASE_URL + file_name
        save_path = os.path.join(SAVE_DIR, file_name)

        # Skip if already downloaded
        if os.path.exists(save_path):
            print(f"Already downloaded: {file_name}")
            continue

        try:
            response = requests.get(file_url, stream=True)
            if response.status_code == 200:
                with open(save_path, 'wb') as f:
                    for chunk in tqdm(response.iter_content(chunk_size=8192), desc=f"Downloading {file_name}", unit="KB", leave=False, position=2):
                        f.write(chunk)
            else:
                print(f"File not found (maybe doesn't exist): {file_name}")
        except Exception as e:
            print(f"Error downloading {file_name}: {e}")


Years:   0%|          | 0/16 [00:00<?, ?year/s]

Months:   0%|          | 0/12 [00:00<?, ?month/s]

Months:   0%|          | 0/12 [00:00<?, ?month/s]

Months:   0%|          | 0/12 [00:00<?, ?month/s]

Months:   0%|          | 0/12 [00:00<?, ?month/s]

...

Months:   0%|          | 0/12 [00:00<?, ?month/s]

Months:   0%|          | 0/12 [00:00<?, ?month/s]

Months:   0%|          | 0/12 [00:00<?, ?month/s]

Months:   0%|          | 0/12 [00:00<?, ?month/s]

Months:   0%|          | 0/12 [00:00<?, ?month/s]

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.7357,
	"longitude": -74.1724,
	"start_date": "2009-04-27",
	"end_date": "2025-05-11",
	"hourly": ["rain", "snowfall", "snow_depth", "precipitation", "apparent_temperature", "relative_humidity_2m", "temperature_2m", "dew_point_2m"],
	"timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_rain = hourly.Variables(0).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(1).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(4).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(5).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(6).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["precipitation"] = hourly_precipitation
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)
